In [1]:
from numpy.fft import fft
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [2]:
N_data = 2500  # Number of training data in final augmented dataset
L = 1024      # Length of IQ sequence in each sample
D = 100 # Decimation factor for long-term data
SNRdBmin = 10
SNRdBmax = 20

max_pw  = 1000 # Pulse Width (us)
max_pri = 1000 # Pulse Repetition Interval (us)
SNRdBmin = 0
SNRdBmax = 20
fs = 20e6

In [3]:
def gen_data():
    data = np.zeros((N_data, L, 8))
    labels = np.zeros((N_data, 2)) # 2 columns: 0 is pulse width, 1 is PRI

    for i in range(N_data):
        # Select Radar emitter parameters
        pri = random.uniform(0, np.log10(max_pri)) # PRI needs to be at least as long as the pulse width, or the radar will be CW
        pri = 10**pri
        pw = random.uniform(0, np.log10(min(pri, max_pw)))
        pw = 10**pw
        phase_off = random.uniform(0, 2*np.pi)

        pulse_phase = random.uniform(0, 1) # Phase between 0 and 1 as percent of cycle elapsed at start of waveform
        mod_type = 0

        # t_short = np.linspace(0.0, 1023/fs, 1024)
        # t_long = 10*t_short
        t = np.linspace(0.0, (L*D-1)/fs, L*D)

        # mag_short = ((t_short + pulse_phase*pri) % (pri*1e-6)) < (pw*1e-6)
        # mag_long =  ((t_long  + pulse_phase*pri) % (pri*1e-6)) < (pw*1e-6)
        mag =  ((t + pulse_phase*pri) % (pri*1e-6)) < (pw*1e-6)

        # x_short = mag_short*np.exp(1j*phase_off)
        # x_long  = mag_long*np.exp(1j*phase_off)
        x = mag*np.exp(1j*phase_off)

        # Add AWGN noise with random power
        SNRdB = random.uniform(SNRdBmin, SNRdBmax)
        SNR = 10**(SNRdB/10)
        sig_pwr = np.mean(np.abs(x)**2)
        sigma2 = sig_pwr/SNR
        noise = np.random.randn(np.size(x))*np.sqrt(sigma2/2) + 1j*np.random.randn(np.size(x))*np.sqrt(sigma2/2)
        x_long = x + noise

        x_short = x_long[np.arange(0, int(np.size(x_long)/D), 1)]
        x_long = x_long[np.arange(0, np.size(x_long), D)]

        xf_short = fft(x_short)
        xf_long  = fft(x_long)


        x_ms = abs(x_short)
        x_ps = np.angle(x_short)
        x_ml = abs(x_long)
        x_pl = np.angle(x_long)
        xf_ms = abs(xf_short)
        xf_ps = np.angle(xf_short)
        xf_ml = abs(xf_long)
        xf_pl = np.angle(xf_long)

        data[i,:,0] = x_ms
        data[i,:,1] = x_ps
        data[i,:,2] = x_ml
        data[i,:,3] = x_pl
        data[i,:,4] = xf_ms
        data[i,:,5] = xf_ps
        data[i,:,6] = xf_ml
        data[i,:,7] = xf_pl

        labels[i,0] = pw
        labels[i,1] = pri

    return data, labels


In [4]:
x, y = gen_data()

In [ ]:
def build_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='relu'))
    return model